In [ ]:
import cv2
import cupy as cp
import numpy as np
import os
from tqdm import tqdm

original_folder = "..."
output_folder = "filtered_videos"

def create_output_dirs():
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(os.path.join(output_folder, "..."), exist_ok=True)

def apply_low_pass_filter_gpu(img_batch, cutoff=45):
    batch_size, height, width, _ = img_batch.shape
    filtered_batch = []

    for img in img_batch:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_gpu = cp.asarray(gray)

        dft = cp.fft.fft2(gray_gpu)
        dft_shift = cp.fft.fftshift(dft)

        crow, ccol = height // 2, width // 2
        mask = cp.zeros((height, width), dtype=cp.float32)
        y, x = cp.ogrid[:height, :width]
        mask_area = (x - ccol)**2 + (y - crow)**2 <= cutoff**2
        mask[mask_area] = 1

        filtered_dft = dft_shift * mask
        f_ishift = cp.fft.ifftshift(filtered_dft)
        img_back = cp.fft.ifft2(f_ishift)
        img_back = cp.abs(img_back)

        img_back = cp.asnumpy(img_back)
        img_back = cv2.normalize(img_back, None, 0, 255, cv2.NORM_MINMAX)
        img_back = np.uint8(img_back)
        filtered_batch.append(cv2.cvtColor(img_back, cv2.COLOR_GRAY2BGR))

    return np.array(filtered_batch)

def process_video(input_video_path, output_video_path, cutoff=45, batch_size=16):
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Could not open {input_video_path}")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')

    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames = []
    for _ in tqdm(range(frame_count), desc=f"Processing {os.path.basename(input_video_path)}"):
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

        if len(frames) == batch_size:
            filtered_frames = apply_low_pass_filter_gpu(np.array(frames), cutoff)
            for filtered_frame in filtered_frames:
                out.write(filtered_frame)
            frames = []

    if frames:
        filtered_frames = apply_low_pass_filter_gpu(np.array(frames), cutoff)
        for filtered_frame in filtered_frames:
            out.write(filtered_frame)

    cap.release()
    out.release()
    print(f"Filtered video saved: {output_video_path}")

def main(cutoff=45, batch_size=16):
    create_output_dirs()
    print("Processing Videos...")
    for file in os.listdir(original_folder):
        if file.endswith(('.mp4', '.avi', '.mov')):
            input_video_path = os.path.join(original_folder, file)
            output_video_path = os.path.join(output_folder, "...", f"{file}")
            process_video(input_video_path, output_video_path, cutoff=cutoff, batch_size=batch_size)

if __name__ == "__main__":
    main(cutoff=45, batch_size=16)


Processing Videos...


Processing 000_003.mp4: 100%|██████████| 396/396 [00:09<00:00, 42.24it/s]


Filtered video saved: filtered_videos\FaceShifter\000_003.mp4


Processing 001_870.mp4: 100%|██████████| 460/460 [00:26<00:00, 17.48it/s]


Filtered video saved: filtered_videos\FaceShifter\001_870.mp4


...

Filtered video saved: filtered_videos\FaceShifter\997_040.mp4


Processing 998_561.mp4: 100%|██████████| 344/344 [00:36<00:00,  9.41it/s]


Filtered video saved: filtered_videos\FaceShifter\998_561.mp4


Processing 999_960.mp4: 100%|██████████| 335/335 [00:14<00:00, 22.63it/s]


Filtered video saved: filtered_videos\FaceShifter\999_960.mp4
